<a href="https://colab.research.google.com/github/0xc3nt4ur10n/TorrentToGDrive/blob/main/TorrentToGDrive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <font color='0f81c1'>__Torrent to Google Drive Downloader__

### <font color='0f81c1'>Install Dependency

In [ ]:
#@markdown <h3>Step #1. Install libtorrent and Initialize Session

try:
    print("🔄 Updating pip, setuptools, and wheel...")
    !python -m pip install --upgrade pip setuptools wheel
except Exception as e:
    print(f"⚠️ Error updating pip, setuptools, or wheel: {e}")

try:
    print("\n🔄 Installing lbry-libtorrent...")
    !python -m pip install lbry-libtorrent
except Exception as e:
    print(f"\n⚠️ Error installing lbry-libtorrent: {e}")

try:
    import libtorrent as lt
    ses = lt.session()
    ses.listen_on(6881, 6891)
    print("\n🚀 Libtorrent session started!")
except Exception as e:
    print(f"\n⚠️ Error initializing libtorrent session: {e}")

### <font color='0f81c1'>Mount Google Drive

In [ ]:
#@markdown <h3>Step #2. Mount Google Drive

from google.colab import drive
import os

# Define mount point
mount_point = "/content/drive"

# Mount Google Drive
try:
    print("🔄 Mounting Google Drive...")
    drive.mount(mount_point)

    # Verify if the mount point exists
    if os.path.ismount(mount_point):
        print("\n✅ Google Drive successfully mounted!")
    else:
        raise Exception("\n❌ Google Drive mount failed. The mount point does not exist.")

except Exception as e:
    print(f"\n⚠️ Error mounting Google Drive: {e}")

## <font color='0f81c1'>Select Save Path Location

In [ ]:
#@markdown <h3>Step #3. Select Download Location

import os

# Step 3: Function to select the download save path
def select_save_path():
    print("\n📂 Select a folder for saving torrents:")
    print("1. Movies")
    print("2. Music")
    print("3. Books")
    print("4. Documents")
    print("5. Adults")
    print("6. Others (custom)")

    choice = input("🔢 Enter the number of your choice (1-6): ")

    # Map choices to save paths
    save_paths = {
        '1': 'Movies',
        '2': 'Music',
        '3': 'Books',
        '4': 'Documents',
        '5': 'Adults'
    }

    # If choice is 6, prompt for a custom folder name
    if choice == '6':
        custom_folder = input("📁 Enter a custom folder name: ")
        final_save_path = f'/content/drive/MyDrive/Torrents/{custom_folder}'
    elif choice in save_paths:
        selected_path = save_paths[choice]
        final_save_path = f'/content/drive/MyDrive/Torrents/{selected_path}'
    else:
        print("❌ Invalid selection. Please try again.")
        return select_save_path()  # Recursively prompt until valid input

    # Check if the path exists; if not, create it
    if not os.path.exists(final_save_path):
        print(f"❌ The folder does not exist. Creating the folder: {final_save_path}")
        os.makedirs(final_save_path, exist_ok=True)

    print(f"\n✅ Save path selected: {final_save_path}")
    return final_save_path

# In Step 3 cell, just call select_save_path() to return the save path.
save_path = select_save_path()  # This will be passed to the next step.

## <font color='0f81c1'>Add Torrent Source



In [ ]:
#@markdown <h3>Step #4. Add From Magnet Link or Torrent File

import libtorrent as lt
from google.colab import files

# Initialize list of downloads
downloads = []

# Function to determine the file type from the torrent
def get_file_icon_from_torrent(download):
    try:
        # Wait for metadata to be downloaded
        if not download.has_metadata():
            return "⏳"  # Show a waiting icon if metadata is not yet available

        # If metadata is available, proceed with extracting the file info
        ti = download.get_torrent_info()

        # Check if the torrent contains multiple files or a single file
        if ti.num_files() > 1:
            return "📂"  # Multi-file torrent (folder)
        else:
            filename = ti.files()[0].path.lower()

            # Check the file extension and return appropriate icon
            if filename.endswith(('.mp4', '.mkv', '.avi', '.mov')):
                return "🎬"  # Movie or video file
            elif filename.endswith(('.mp3', '.flac', '.wav')):
                return "🎵"  # Music file
            elif filename.endswith(('.pdf', '.docx', '.epub')):
                return "📚"  # Book file
            elif filename.endswith(('.txt', '.csv')):
                return "📄"  # Document or text file
            else:
                return "📦"  # Generic file/package
    except Exception as e:
        print(f"⚠️ Error retrieving torrent info: {e}")
        return "📦"  # Default if no info or an error occurs

# Add torrents from magnet links or .torrent files
def add_torrent_to_download(save_path):
    while True:
        source = input("🔗 Enter a magnet link or type 'file' to upload a .torrent file or 'exit' to stop: ")

        if source.lower() == 'exit':
            print("\n👋 Exiting input mode.")
            break

        elif source.lower() == 'file':
            print("📁 Uploading a .torrent file...")
            uploaded = files.upload()
            if uploaded:
                torrent_file = list(uploaded.keys())[0]
                try:
                    info = lt.torrent_info(torrent_file)
                    h = ses.add_torrent({'ti': info, 'save_path': save_path})
                    downloads.append(h)
                    icon = get_file_icon_from_torrent(h)
                    print(f"\n🆕 Starting: {icon} {h.status().name}")
                except Exception as e:
                    print(f"⚠️ Error loading torrent: {e}")
            else:
                print("⚠️ No file uploaded. Please try again.")

        elif source.startswith("magnet:?xt="):
            print("\n🔗 Adding magnet link...")
            try:
                params = lt.parse_magnet_uri(source)
                params.save_path = save_path
                h = ses.add_torrent(params)
                downloads.append(h)
                icon = get_file_icon_from_torrent(h)
                print(f"\n🆕 Starting: {icon} {h.status().name}")
            except Exception as e:
                print(f"⚠️ Invalid magnet link: {e}")
        else:
            print("❌ Invalid input. Please enter a valid magnet link or type 'file' to upload a torrent file.")

add_torrent_to_download(save_path)

## <font color='0f81c1'>Start Download

In [ ]:
#@markdown <h3>Step #5. Start Download

import time
from IPython.display import clear_output

# Lists
ongoing_downloads = downloads
completed_downloads = []

# ⏳ Monitor downloads
while ongoing_downloads:
    clear_output(wait=True)
    next_iteration_downloads = []
    display_list = []

    # Collect display strings and organize
    for download in ongoing_downloads:
        s = download.status()

        # 🌀 State strings
        state_str = [
            'queued',
            'checking',
            'downloading metadata',
            'downloading',
            'finished',
            'seeding',
            'allocating',
            'checking fastresume'
        ]
        state = state_str[s.state] if s.state < len(state_str) else "unknown"

        icon = get_file_icon_from_torrent(download)

        # ETA calculation with hours
        if s.download_rate > 0 and s.total_wanted > 0:
            eta_seconds = (s.total_wanted - s.total_done) / s.download_rate
            hours = int(eta_seconds) // 3600  # Calculate hours
            minutes = (int(eta_seconds) % 3600) // 60  # Calculate minutes
            seconds = int(eta_seconds) % 60  # Calculate remaining seconds

            if hours > 0:
                eta_str = f"ETA: {hours}h {minutes}m {seconds}s"
            elif minutes > 0:
                eta_str = f"ETA: {minutes}m {seconds}s"
            else:
                eta_str = f"ETA: {seconds}s"
        else:
            eta_str = "ETA: ∞"

        if s.is_seeding:
            display_list.insert(0, f'{icon} {s.name} ✅ {s.progress * 100:.2f}% complete! 🌀 seeding ⬇️ 0.0 kB/s ⬆️ {s.upload_rate / 1000:.1f} kB/s 👥 peers: {s.num_peers}')
            completed_downloads.append(download)
        else:
            display_list.append(f'{icon} {s.name} ⬇️ {s.progress * 100:.2f}% 🌀 {state} ⏳ {eta_str} ⬇️ {s.download_rate / 1000:.1f} kB/s ⬆️ {s.upload_rate / 1000:.1f} kB/s 👥 peers: {s.num_peers}')
            next_iteration_downloads.append(download)

    # Display the download status in sorted order
    for line in display_list:
        print(line)

    # Update ongoing downloads
    ongoing_downloads = next_iteration_downloads
    time.sleep(1)